<a href="https://colab.research.google.com/github/bibleme/Aiary/blob/one_line_diary/%ED%95%98%EB%A3%A8%EC%9D%BC%EA%B8%B0_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================
# AIARY Inference (v2)
# 파일 내 여러 "한 줄 일기" 전체를 합쳐서
# 하루 일기 1개 생성하는 버전
# ============================================

from google.colab import drive
drive.mount('/content/drive')

!pip install -q "transformers>=4.36.0,<5" sentencepiece accelerate emoji regex pandas

import os
import json
import torch
import regex as re
import emoji
import pandas as pd
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast

# ---------------------------------------
# 1) 경로 설정
# ---------------------------------------
BASE_DIR = "/content/drive/MyDrive/aiary"

MODEL_DIR = os.path.join(BASE_DIR, "models/day_diary_from_summary_v2")
ONE_LINE_FILE = os.path.join(BASE_DIR, "outputs/one_line_texts.txt")

OUTPUT_JSON = os.path.join(BASE_DIR, "outputs/generated_diary_from_one_line.json")
OUTPUT_TXT  = os.path.join(BASE_DIR, "outputs/generated_diary_from_one_line.txt")

MAX_INPUT_LEN = 256
MAX_TARGET_LEN = 220


# ---------------------------------------
# 2) 이모지 제거 + 텍스트 정제 함수
# ---------------------------------------
def remove_emoji(text: str) -> str:
    text = emoji.replace_emoji(text, "")
    emoji_pattern = re.compile(r"[\p{Emoji}\p{Emoji_Presentation}\p{Extended_Pictographic}]", flags=re.UNICODE)
    return emoji_pattern.sub("", text)

def clean_sentence(text: str) -> str:
    text = str(text)
    text = remove_emoji(text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# ---------------------------------------
# 3) 모델 / 토크나이저 로드
# ---------------------------------------
print("[INFO] 모델 / 토크나이저 로드 중...")
if not os.path.exists(MODEL_DIR):
    raise FileNotFoundError(f"모델 디렉토리를 찾을 수 없습니다: {MODEL_DIR}")

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_DIR)
model = BartForConditionalGeneration.from_pretrained(MODEL_DIR)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print(f"[INFO] device = {device}")
print("[INFO] 모델 로드 완료")


# ---------------------------------------
# 4) 요약(여러 줄) → 하루일기 생성 함수
# ---------------------------------------
def generate_diary_from_summary(summary_text: str,
                                max_len: int = MAX_TARGET_LEN) -> str:
    """
    summary_text: "1. ~\n2. ~\n3. ~"처럼 여러 줄로 된 요약 전체
    return: KoBART가 생성한 하루 일기 텍스트
    """

    input_text = f"[SUMMARY]\n{summary_text}\n[DIARY]"

    enc = tokenizer(
        input_text,
        max_length=MAX_INPUT_LEN,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_len,
            min_length=40,
            no_repeat_ngram_size=3,
            repetition_penalty=2.0,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            early_stopping=True,
            eos_token_id=tokenizer.eos_token_id,
        )

    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred = pred.replace("[DIARY]", "").strip()
    return pred


# ---------------------------------------
# 5) one_line_texts.txt 로드 (전체가 하루 인풋)
# ---------------------------------------
if not os.path.exists(ONE_LINE_FILE):
    raise FileNotFoundError(f"한 줄 일기 파일을 찾을 수 없습니다: {ONE_LINE_FILE}")

with open(ONE_LINE_FILE, "r", encoding="utf-8") as f:
    raw_lines = [line.strip() for line in f.readlines() if line.strip()]

if not raw_lines:
    raise ValueError("one_line_texts.txt 안에 내용이 없습니다.")

print(f"[INFO] 파일에서 읽은 문장 개수: {len(raw_lines)}")
print("[LOG] 원본 한 줄 일기들:")
for ex in raw_lines:
    print("-", ex)

# 이모지 제거 + 공백 정리
cleaned_sentences = [clean_sentence(s) for s in raw_lines]

# "1. ~" 형식으로 번호 붙여서 summary 구성
bullet_lines = []
for i, sent in enumerate(cleaned_sentences, start=1):
    # 이미 '1.' 같이 번호로 시작하면 그대로 둘 수도 있지만,
    # 여기서는 무조건 새로 번호 붙이는 게 깔끔함
    bullet_lines.append(f"{i}. {sent}")

combined_summary = "\n".join(bullet_lines)

print("\n[LOG] 모델에 들어갈 SUMMARY 포맷:")
print(combined_summary)


# ---------------------------------------
# 6) 하루일기 생성
# ---------------------------------------
print("\n[INFO] 하루 일기 생성 중...")
generated_diary = generate_diary_from_summary(combined_summary)
print("\n[GENERATED DIARY]")
print(generated_diary)


# ---------------------------------------
# 7) 결과 저장 (JSON + TXT)
# ---------------------------------------
result_obj = {
    "raw_lines": raw_lines,                # 원본 한 줄 일기들 (이모지 포함)
    "cleaned_summary_bullets": bullet_lines,  # "1. ~" 형식 summary
    "combined_summary": combined_summary,
    "generated_diary": generated_diary,
}

os.makedirs(os.path.dirname(OUTPUT_JSON), exist_ok=True)

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(result_obj, f, ensure_ascii=False, indent=2)

with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
    f.write(generated_diary + "\n")

print(f"\n[INFO] JSON 저장 완료: {OUTPUT_JSON}")
print(f"[INFO] TXT 저장 완료:  {OUTPUT_TXT}")
print("\n[INFO] 인퍼런스 전체 완료")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


[INFO] 모델 / 토크나이저 로드 중...
[INFO] device = cpu
[INFO] 모델 로드 완료
[INFO] 파일에서 읽은 문장 개수: 5
[LOG] 원본 한 줄 일기들:
- 따뜻한 차 안에서 과자 먹방 시작🍪🚗💕
- 고기 냄새 가득한 따뜻한 가족 외식시간🍖🥰
- 장난감 총과 친구, 오늘도 신나게 놀았어요🔫🤠✨
- 따끈한 고기 냄새에 기대 가득한 밤🍖✨
- 불꽃 속에서 아빠 손 잡고 따뜻함을 느꼈어요🔥🤗

[LOG] 모델에 들어갈 SUMMARY 포맷:
1. 따뜻한 차 안에서 과자 먹방 시작
2. 고기 냄새 가득한 따뜻한 가족 외식시간
3. 장난감 총과 친구, 오늘도 신나게 놀았어요
4. 따끈한 고기 냄새에 기대 가득한 밤
5. 불꽃 속에서 아빠 손 잡고 따뜻함을 느꼈어요

[INFO] 하루 일기 생성 중...

[GENERATED DIARY]
오늘 아침 차 안에서 과자 먹방 했어. 고기와 향이 진하게 퍼지면서, 나도 덩달아 기분이 좋았네. 이렇게 따뜻한 날이 언제까지나 계속되면 좋겠어! 오늘도 아빠 손 잡고 따뜻해졌거든. 불꽃놀이 하는 모습을 보며 다시 힘을 내서 함께 놀아보려고 해.

오늘은 정말 행복했네!
따뜻한 하루였는데, 이제 이 순간을 영원히 간직하고 싶어져요 "꺅!" 하며 신나게 뛰어다녔어.
밤에 창밖으로 햇살이 들어와서, 아늑함이 더해져 더욱 평화로운 시간이었지. 그 작은 손으로 공원에서 키운 꽃들이 빛나고 있었으니 너무 기뻤고, 앞으로도 이런 소중한 시간이 많았으면 좋겠다는 생각이 들어요. 
꽃잎처럼 빛나는 밤에 너와 함께하는 시간을 소중히 만들어주길 바라며, 오늘은 꼭 크리스마스 선물을 준비할게요.
육아는 힘들지만, 사랑이 가득한 날들을 기억해두고 싶어서 마음이 다차분해지더라구요
"
또 한 번 도전하는 게 가장 중요하다는 걸 느꼈어, 육아의 어려움 속에서도 사랑을 느끼는 것 같아 매일매일 감사해야겠다. 가족들과

[INFO] JSON 저장 완료: /content/drive/MyDrive/aiary/outputs/generated_